In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import plotly.express as px
from tensorflow.keras import layers
import tensorflow as tf
from sklearn.model_selection import train_test_split

from sklearn.manifold import TSNE

In [8]:
df = pd.read_json('./train.json', lines=True).drop('index', axis=1)
df = df.query("signal_to_noise >= 1")
test_df = pd.read_json('./test.json', lines=True).drop('index', axis=1)

# preprocessing

measured_cols = ['reactivity_error', 'deg_error_Mg_pH10', 'deg_error_pH10', 'deg_error_Mg_50C',
            'deg_error_50C', 'reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']

for col in measured_cols:
    df[col] = df[col].apply(lambda x: np.array(x))

# visualizations

pred_cols = ['reactivity','deg_Mg_pH10','deg_pH10','deg_50C','deg_Mg_50C']

input_cols = ['sequence', 'structure', 'predicted_loop_type']


# Feature Extraction

- front, middle, end flag ( what is the placement of the amino in its codon )

### TODO

- create codon mapping opposed to character mapping

- ammend character mapping to remove bias introduced by the simple application of enumerated mapping

- experiment with one hot encoding and subsequent decomposition


In [9]:

def create_input(temp):
    
    # create features
    
    map_dict = {'(': 0, ')': 1, '.': 2, 'A': 3, 'C': 4, 'G': 5, 'U': 6, 'B': 7,
              'E': 8, 'H': 9, 'I': 10, 'M': 11, 'S': 12, 'X': 13}
    
    for col in ['sequence', 'structure', 'predicted_loop_type']:
        temp[col] = temp[col].apply(lambda x: np.array([map_dict[i] for i in x[:68]]))
  
    def amino_pos(x):
        pos_list = []
        
        for i,v in enumerate(x):
            if i % 3 == 0:
                pos_list.append('start')
            if i % 3 == 1:
                pos_list.append('middle')
            if i % 3 == 2:
                pos_list.append('end')
                
        return pos_list

    temp['position'] = np.nan
    temp.position = temp.sequence.apply(amino_pos)
    
    for col in ['reactivity_error','deg_error_Mg_pH10','deg_error_pH10','deg_error_Mg_50C','deg_error_50C']:
        temp[col] = temp[col].apply(lambda x: np.mean(x))
        
    temp = temp[(((temp.reactivity_error >= 0) & (temp.reactivity_error <= 0.4)) & 
                    ((temp.deg_error_Mg_pH10 >= 0) & (temp.deg_error_Mg_pH10 <= 0.4)) & 
                   ((temp.deg_error_pH10 >= 0) & (temp.deg_error_pH10 <= 0.4)) & 
                   ((temp.deg_error_Mg_50C >= 0) & (temp.deg_error_Mg_50C <= 0.4)) & 
                   ((temp.deg_error_50C >= 0) & (temp.deg_error_50C <= 0.4)))]
    
    # Start exploding
    
    storage = temp.copy().drop('sequence', axis=1)
    
    holder = temp.explode('sequence').drop(['structure', 'predicted_loop_type'], axis=1)

    del temp
    
    for col in ['structure', 'predicted_loop_type','corrected_reactivity','corrected_deg_Mg_pH10','corrected_deg_pH10',
             'corrected_deg_50C','corrected_deg_Mg_50C', 'position']:
        holder[col] = storage[col].explode(col).values
        
    holder = holder.reset_index(drop=True)

    holder = pd.concat([holder,pd.get_dummies(holder['position'])], axis=1)
    
    holder = holder[pred_cols+['sequence', 'structure', 'predicted_loop_type','start','middle','end']].astype(float)

    train_inputs = holder[['sequence', 'structure', 'predicted_loop_type','start','middle','end']].values.reshape(len(storage),68,6)
    train_labels = holder[pred_cols].values.reshape(len(storage),68,5)

    return train_inputs, train_labels

In [10]:
train_inputs, train_labels = create_input(df)

In [11]:
def create_test_input(temp, seq_len):
    # create features
    
    map_dict = {'(': 0, ')': 1, '.': 2, 'A': 3, 'C': 4, 'G': 5, 'U': 6, 'B': 7,
              'E': 8, 'H': 9, 'I': 10, 'M': 11, 'S': 12, 'X': 13}
    
    for col in ['sequence', 'structure', 'predicted_loop_type']:
        temp[col] = temp[col].apply(lambda x: np.array([map_dict[i] for i in x]))

#     temp['order'] = np.nan
    
#     def list_len(x):
#         return np.arange(1,len(x.sequence)+1,1)
    
#     temp.order = temp.apply(list_len, axis=1)
    
    def amino_pos(x):
        pos_list = []
        
        for i,v in enumerate(x):
            if i % 3 == 0:
                pos_list.append('start')
            if i % 3 == 1:
                pos_list.append('middle')
            if i % 3 == 2:
                pos_list.append('end')
                
        return pos_list

    temp['position'] = np.nan
    temp.position = temp.sequence.apply(amino_pos)
    
    # Start exploding
    
    storage = temp.copy().drop('sequence', axis=1)
    
    holder = temp.explode('sequence').drop(['structure', 'predicted_loop_type'], axis=1)

    del temp
    
    for col in ['structure', 'predicted_loop_type','position']:
        holder[col] = storage[col].explode(col).values
        
    holder = holder.reset_index(drop=True)
    
    holder = pd.concat([holder,pd.get_dummies(holder['position'])], axis=1)
    
    holder = holder[['sequence', 'structure', 'predicted_loop_type','start','middle','end']].astype(float)

    train_inputs = holder[['sequence', 'structure', 'predicted_loop_type','start','middle','end']].values.reshape(len(storage),seq_len,6)
    
    return train_inputs

# Build Bi-directional GRU with MCRMSE as loss function

- LSTM had no benifit, would collapse after a few epochs with a loss of astronomical proportions.

- SimpleRNN provided no additional insight over GRU

- GRU with exponential linear unit activation performed best to date. Initially layers were 'too far' apart, closed the gap by introducing new layers. with smaller step sizes between them. This helped get below 0.3 MCRMSE. 

- Want to experiment with more feed forward layers amongst the GRU layers, as well as other kernel initializations. Although, I am not sure any would outperform the orthagonal initilization. Starting with zeros and increasing epochs, may come close to or perform as well, I think. 

In [35]:
def MCRMSE(y_true, y_pred):
    colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=1)
    return tf.reduce_mean(tf.sqrt(colwise_mse), axis=1)

def build_model(seq_len, pred_len, feats_len, embed_size=108, dropout=0.25, sp_dropout=0.25,  embed_dim=136):
    
    inputs = layers.Input(shape=(seq_len, feats_len))

    embed = layers.Embedding(input_dim=embed_size, output_dim=embed_dim)(inputs)

    hidden = tf.reshape(
        embed, shape=(-1, embed.shape[1],  embed.shape[2] * embed.shape[3])
    )

#     hidden = layers.SpatialDropout1D(sp_dropout)(hidden)

    hidden = layers.Bidirectional(layers.GRU(
            200, activation='elu', dropout=dropout, return_sequences=True, kernel_initializer='orthogonal'))(hidden)
    
    hidden = layers.Bidirectional(layers.GRU(
            300, activation='elu', dropout=dropout, return_sequences=True, kernel_initializer='orthogonal'))(hidden)
    
    hidden = layers.Bidirectional(layers.GRU(
            500, activation='elu', dropout=dropout, return_sequences=True, kernel_initializer='orthogonal'))(hidden)
    
    hidden = layers.Bidirectional(layers.GRU(
            300, activation='elu', dropout=dropout, return_sequences=True, kernel_initializer='orthogonal'))(hidden)
    
    hidden = layers.Bidirectional(layers.GRU(
            150, activation='elu', dropout=dropout, return_sequences=True, kernel_initializer='orthogonal'))(hidden)
    
    # Since we are only making predictions on the first part of each sequence, 
    # we have to truncate it
#     truncated = hidden[:, :pred_len]
#     truncated = layers.Dense(68, activation='elu')(hidden)

    out = layers.Dense(5, activation='elu')(hidden)

    model = tf.keras.Model(inputs=inputs, outputs=out)
    model.compile(tf.optimizers.Adam(lr=0.001), loss=MCRMSE)

#     model.summary()
    
    return model




In [36]:
%%time

x_train, x_val, y_train, y_val = train_test_split(train_inputs, train_labels, 
                                                  test_size=.2, random_state=0)

model = build_model(68, 68, train_inputs.shape[-1])

history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    batch_size=100,
    epochs=60,
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',patience=2),
        tf.keras.callbacks.ModelCheckpoint('model.h5')
    ]
)

Train on 1581 samples, validate on 396 samples
Epoch 1/60
1581/1581 [==============================] - 31s 20ms/sample - loss: 0.6537 - val_loss: 0.4561
Epoch 2/60
1581/1581 [==============================] - 21s 13ms/sample - loss: 0.4577 - val_loss: 0.3932
Epoch 3/60
1581/1581 [==============================] - 34s 22ms/sample - loss: 0.4100 - val_loss: 0.3772
Epoch 4/60
1581/1581 [==============================] - 57s 36ms/sample - loss: 0.3921 - val_loss: 0.3702
Epoch 5/60
1581/1581 [==============================] - 21s 13ms/sample - loss: 0.3831 - val_loss: 0.3624
Epoch 6/60
1581/1581 [==============================] - 20s 13ms/sample - loss: 0.3777 - val_loss: 0.3585
Epoch 7/60
1581/1581 [==============================] - 20s 13ms/sample - loss: 0.3717 - val_loss: 0.3540
Epoch 8/60
1581/1581 [==============================] - 26s 16ms/sample - loss: 0.3622 - val_loss: 0.3465
Epoch 9/60
1581/1581 [==============================] - 56s 36ms/sample - loss: 0.3549 - val_loss: 0.3435

In [38]:
fig = px.line(
    history.history, y=['loss', 'val_loss'],
    labels={'index': 'epoch', 'value': 'MCRMSE'}, 
    title='Training History')

fig.show()

In [39]:
# Caveat: The prediction format requires the output to be the same length as the input,
# although it's not the case for the training data.
model_public = build_model(107, 107, train_inputs.shape[-1])
model_private = build_model(130, 130, train_inputs.shape[-1])

model_public.load_weights('model.h5')
model_private.load_weights('model.h5')

public_df = test_df.query("seq_length == 107")
private_df = test_df.query("seq_length == 130")

public_inputs = create_test_input(public_df, 107)
private_inputs = create_test_input(private_df, 130)

public_preds = model_public.predict(public_inputs)
private_preds = model_private.predict(private_inputs)

C:\Users\ASilva2\AppData\Local\Continuum\miniconda3\envs\covid\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ASilva2\AppData\Local\Continuum\miniconda3\envs\covid\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ASilva2\AppData\Local\Continuum\miniconda3\envs\covid\lib\site-packages\pandas\core\generic.py:5165: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [40]:
preds_ls = []

for df, preds in [(public_df, public_preds), (private_df, private_preds)]:
    for i, uid in enumerate(df.id):
        single_pred = preds[i]

        single_df = pd.DataFrame(single_pred, columns=pred_cols)
        single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]

        preds_ls.append(single_df)

preds_df = pd.concat(preds_ls)

print(preds_df.shape)

preds_df.to_csv('./submission_acs.csv', index=False)